In [1]:
!pip install wandb datasets transformers

In [2]:
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from dataclasses import dataclass
from tokenizers import Tokenizer
from pathlib import Path

from torch.utils.data import Dataset, DataLoader
import wandb
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
!wandb login

In [ ]:
wandb.init(
    project = "DPO IMPLEMENTATION"
)

In [5]:
# defining hyperparameters(taken from paper)

batch_size = 1
beta = 0.5#(increasing beta will make the model generate preferred respsonses more)
max_lr = 1e-6

In [6]:
class DPO:
  def __init__(self, ref_model, sft_model, device, beta, tokenizer):


    self.ref_model = ref_model
    self.sft_model = sft_model
    self.device=device
    self.beta = beta
    self.tokenizer = tokenizer
    self.ref_model.eval()



  def DPOloss(self, datapoint):



    self.win_prompt = datapoint['chosen']
    self.lose_prompt = datapoint['rejected']

    with torch.no_grad():
      self.win_log_ref = torch.nn.functional.log_softmax(self.ref_model(**self.win_prompt).logits, dim=-1)
      self.win_log_ref = torch.gather(self.win_log_ref, -1, self.win_prompt['input_ids'].unsqueeze(-1)).squeeze(-1) #Why gather? Because its not token level stuff we care about but se
      self.win_log_ref = self.win_log_ref * (self.win_prompt['attention_mask'])
      
      
      self.lose_log_ref = torch.nn.functional.log_softmax(self.ref_model(**self.lose_prompt).logits, dim=-1)
      self.lose_log_ref = torch.gather(self.lose_log_ref, -1, self.lose_prompt['input_ids'].unsqueeze(-1)).squeeze(-1) #Why gather? Because its not token level stuff we care about but se
      self.lose_log_ref = self.lose_log_ref * (self.lose_prompt['attention_mask'])
      
    self.win_log_sft = torch.nn.functional.log_softmax(self.sft_model(**self.win_prompt).logits, dim=-1)
    self.win_log_sft = torch.gather(self.win_log_sft, -1, self.win_prompt['input_ids'].unsqueeze(-1)).squeeze(-1) #Why gather? Because its not token level stuff we care about but se
    self.win_log_sft = self.win_log_sft * (self.win_prompt['attention_mask'])

    self.lose_log_sft = torch.nn.functional.log_softmax(self.sft_model(**self.lose_prompt).logits, dim=-1)
    self.lose_log_sft = torch.gather(self.lose_log_sft, -1, self.lose_prompt['input_ids'].unsqueeze(-1)).squeeze(-1)
    self.lose_log_sft = self.lose_log_sft * (self.lose_prompt['attention_mask'])

    self.diff1 = self.win_log_sft - self.win_log_ref
    self.diff2 = self.win_log_sft - self.lose_log_ref

    self.final = -nn.functional.logsigmoid(self.beta *(self.diff1 - self.diff2)).mean() #Remember we have to maximize the rewards thus minimizing the negative sign! Also, since the var of rewards could be very much, we take mean so as to have a notion of normalizing it!

   
    return self.final


In [7]:
HF_TOKEN =  "HUGGING_FACE_API_KEY"

In [8]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

cuda:0


In [9]:
torch.cuda.set_device(device)

In [10]:
sft_model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-0.5B-Instruct", token=HF_TOKEN, device_map=device)
ref_model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-0.5B-Instruct", token=HF_TOKEN, device_map=device)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-0.5B-Instruct", token=HF_TOKEN, device_map=device)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [11]:
pip install --upgrade accelerate


Note: you may need to restart the kernel to use updated packages.


In [12]:
from datasets import load_dataset, Dataset
dataset = load_dataset("HumanLLMs/Human-Like-DPO-Dataset")
split_dataset = dataset["train"].train_test_split(test_size=0.2, seed=42)
train_dataset = split_dataset["train"]
test_dataset = split_dataset["test"]
print(train_dataset)

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 8707
})


In [13]:
def dpo_collate_fn_merged_prompt(batch):

    merged_chosen_prompts = []
    merged_rejected_prompts = []

    for sample in batch:

       
        prompt = sample['prompt']
        chosen_data = sample['chosen']
        chosen_data = "Instruction: " + prompt + "\n" + "Output: " + chosen_data + "\n"
        
        rejected_data = sample['rejected']
        rejected_data =  "Instruction: " + prompt + "\n" + "Output: " + rejected_data + "\n"

       
        merged_chosen_prompts.append(chosen_data)


        merged_rejected_prompts.append(rejected_data)

    tokenized_win_prompt = tokenizer(merged_chosen_prompts, max_length = 1024, padding='max_length', truncation=True, return_tensors="pt").to(device)

    tokenized_lose_prompt = tokenizer(merged_rejected_prompts, max_length = 1024, truncation=True, padding='max_length', return_tensors="pt").to(device)



    return {
       
        'chosen': tokenized_win_prompt,
        'rejected': tokenized_lose_prompt 
    }
     

In [14]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=dpo_collate_fn_merged_prompt)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, collate_fn=dpo_collate_fn_merged_prompt)

In [15]:
sft_model.train()
optimizer = torch.optim.RMSprop(sft_model.parameters(), lr=max_lr)

total_steps = 4353
eval_iters = 20

dpo_loss = DPO(ref_model, sft_model, device, beta, tokenizer)

@torch.inference_mode()
def estimate_loss():
    out = {}
    sft_model.eval()
    
    for split in ["train", "test"]:
        if split == "train":
            loader = iter(train_loader)  # Reinitialize train iterator
        else:
            loader = iter(test_loader)   # Reinitialize test iterator

        losses = torch.zeros(eval_iters, device=device)

        for k in range(eval_iters):
            try:
                datapoint = next(loader)  # Get next batch
                loss = dpo_loss.DPOloss(datapoint)
                losses[k] = loss.item()
            except StopIteration:
                break  # Stop if dataset is smaller than eval_iters

        out[split] = losses.mean().item()  # Store mean loss

    sft_model.train()
    return out


In [16]:
# from tqdm import tqdm

# train_iterator = iter(train_loader)

# for step in tqdm(range(total_steps)):


#   if (step % eval_iters == 0 and step != 0) or step == total_steps - 1:
#     losses = estimate_loss()
#     print(f"step {step}: train loss {losses['train']:.4f}, val loss {losses['test']:.4f}")
#     wandb.log({
#             "step": step,
#             "training_loss": losses['train'],
#             "val_loss": losses['test']
#         })
    
#     text  = next(train_iterator)
#     loss = dpo_loss.DPOloss(text)
#     optimizer.zero_grad(set_to_none=True)
#     loss.backward()
#     optimizer.step()


In [17]:
from tqdm import tqdm
from torch.cuda.amp import autocast, GradScaler

scaler = GradScaler()  # Gradient scaler for stable FP16 training
train_iterator = iter(train_loader)

for step in tqdm(range(total_steps)):

    if (step % eval_iters == 0 and step != 0) or step == total_steps - 1:
        losses = estimate_loss()
        print(f"step {step}: train loss {losses['train']:.4f}, val loss {losses['test']:.4f}")
        wandb.log({
            "step": step,
            "training_loss": losses['train'],
            "val_loss": losses['test']
        })

    text = next(train_iterator)

    # Enable mixed precision training
    with autocast(dtype=torch.float16):
        loss = dpo_loss.DPOloss(text)

    scaler.scale(loss).backward()  # Scale loss to prevent underflow
    scaler.step(optimizer)  
    scaler.update()  # Update scaler for next iteration
    optimizer.zero_grad(set_to_none=True)


  0%|          | 21/4353 [00:13<3:11:27,  2.65s/it]

step 20: train loss 1.0758, val loss 0.8482


  1%|          | 41/4353 [00:25<3:11:53,  2.67s/it]

step 40: train loss 0.9138, val loss 1.1219


  1%|▏         | 61/4353 [00:37<3:11:56,  2.68s/it]

...

step 4320: train loss 1.0223, val loss 0.9275


100%|█████████▉| 4341/4353 [45:57<00:33,  2.79s/it]

step 4340: train loss 1.0120, val loss 1.0177


100%|██████████| 4353/4353 [46:08<00:00,  1.57it/s]

step 4352: train loss 0.9909, val loss 0.9870


In [20]:
torch.save(sft_model.state_dict(), "dpo_model_weights.pth")